&copy; Copyright for Shuang Wu 2017<br>
Cite from the Deep Learning Book by Ian Goodfellow, Yoshua Bengio and Aaron Courville<br>
Learning notes for myself.

## Convolutional Networks<br>
Applications: time-series data, can be thought of as a 1D grid taking samples at regular time intervals. Image data, can be thought of as a 2D trid of pixels. Convolutional neural network indicates the network employs a mathmatical operation called convolution, which is specialized kind of linear operation.<br>
<strong>Convolutional networks are simply neural networks that use convolution in place of general matrix multiplication in at least one of their layers.</strong>

### The Convolution Operation<br>
Convolution is an operation on two functions of a real-valued argument. Suppose we are tracking the location of a spaceship with a laser. The laser will provide a single output $x(t)$, the position of the spaceship at time $t$. Now the laser have some noisy. To obtain less noisy position info., we like to average together several measurements. Because recent measurements are more relevant, we want this to be a weighted average that gives more weight to recent measurements using a weighting function $w(a)$, a is the age of a measurement. Then we can have new function providing a smoothed estimate of the position:<br>
$$s(t)=\int x(a)w(t-a)da$$<br>
This operation called convolution, typically denoted w/ and asterisk:<br>
$$s(t)=(x*w)(t)$$<br>
Here, $w$ needs to be 0 for all negative arguments and w needs to be a valid probability density function.<br>

In convolutional network terminology, the first argument to the convolution is ofen referred to as the <i>input</i>, x, and the second argument as the <i>kernal</i>, w. The output is sometimes referred to as the <i>feature map</i>.<br>
The discrete convolution equation, which in realistic, the time always been discretized as per second:<br>
$$s(t) = (x*w)(t) = \sum^\infty_{a=-\infty}x(a)w(t-a)$$<br>
In ML application, the input and the kernel are usually multidimensional array of data/parameters, adapted by the learning algorithm. Refer to these multidimensional arrays as tensors. <br>
In practive, we often use convolutions over more than one axis at a time. The equation:<br>
$$S(i,j)=(I*K)(i,j)=\sum_m\sum_n I(m,n)K(i-m, j-n)$$
Or equivalently write:<br>
$$S(i,j)=(I*K)(i,j)=\sum_m\sum_n I(i-m, j-n)K(m, n)$$
For input I and kernal K.<br>

The latter formula is more straightforward to implement in a machine learning library, because less variation in the range of valid values of $m$ and $n$. Many neural network libraries implement a related function called the cross-correlation with the equation:<br>
$$S(i,j)=(I*K)(i,j)=\sum_m\sum_n I(i+m, j+n)K(m, n)$$<br>
This equation same as convolution but w.o flipping the kernei. A exmaple:<br>
![ch91](imgs/ch91.jpg)<br>
This is example of 2-D convolution w/o kernal-flipping.

### Motivation<br>
Convolution leverages 3 important ideas that can help improve a ML system: <i>sparse interactions, parameter sharing, equivariant representations</i>.<br>

Sparse interactions: this is accomplished by making the kernel smaller than the input. When processing an image, the input image might have thousands or millions of pixels, but we can detect small, meaningful features such as edges w/ kernal that  occupy only part of the pixels. This can reduce the parameters, reduces the memory requirements of the model and improves its statistical efficiency. Also requires fewer operations.<br>
![ch92](imgs/ch92.jpg)<br>
Parameter sharing: using the same parameter for more than one function in a model. In CNN, each  member of the kernel is used at every position of the input. The share means that rather than learning a separate set of parameters for every location, we learn only one set. This reduce the storage requirements of the model to k-parameters.<br>

Equivariant: means if the input changes, the output changes in the same way. This is useful for when we know that some function of a small number of neighboring pixels is useful when applied to multiple input locations. When processing images, it is useful to detect edges in the first layer of a convolutional network.

### Pooling<br>
A typical layer of a convolutional network consists of 3 stages.<br>
![ch93](imgs/ch93.jpg)<br>
In first stage, layer performs several convolutions in parallel to produce a set of linear activations. In 2nd stage, each linear activation is run through a nonlinear activation function, like ReLU, named detector stage. In the 3rd stage, use a <i>pooling function</i> to modift the output of the layer.<br>
A pooling function replaces the output of the net at a certain location with a summary statistic of the nearby output. E.g, <strong> max pooling</strong> operation reportthe maximum output w/n a recatangular neighborhood. Other pooling functions include the average of a rectangular neighborhood, the $L^2$ norm of a rectangular neighborhood, or a weighted average based on the distance from the central pixl.<br>
Pooling helps to make the representation become approximately <i>invarint</i> to small translations of the input. Invariance to translation means that if we translate the input by a small amount, most of the value out from the pool not change.
![ch94](imgs/ch94.jpg)<br>
<strong>Invariance to local translation can be a very useful property if we care more about whether some feature is present than exactly where it is.</strong><br>
Pooling over spatial regions produces invariance to translation, but if we pool over the outputs of separately parametrized convolutions, the features can learn which tansformations to become invariant to.<br>
Because pooling summarizes the responses over a whole neighborhood, it is possible to use fewer pooling units than detector units, by reporting summary statistics for poolingregions spaced k pixels aprat rather than 1 pixel apart.This improved statistical effciency and reduced memory requirements for storing the parameters.<br>
For many tasks, pooling is essential for handling inputs of varying size.<br>
![ch95](imgs/ch95.jpg)<br>
![ch96](imgs/ch96.jpg)<br>


### Convolution and Pooling as an Infinitely Strong Prior
An infinitely strong prior places zero probability on some parameters and sya that these parameter values are completely forbidden, regarless of how much support the data gives to hose values.
![ch97](imgs/ch97.jpg)<br>
The use of pooling is an infinitely strong prior that each unit should be invariant to small translations. Though infinitely strong prior would be extremely computationally wasteful with a fully connected net, but it give some insights into how convolutional nets work.<br>
One key insight is that convolution and pooling can cause underfitting. If a task relies on preserving precise spatial information, then pooling on all features can increase thetraining error. Some convolutional network architectures are designed to use pooling on some channel but not on others, in order to get both highly invariant features and features that will not underfit when the translation invariance prior is incorrect.

### Variants of the Basic Convolution Function
Assume we have a 4-D kernel tensor $K$ w/ element $K_{i,j,k,l}$ giving the connection strength between a unit in channel $i$ of the output and a unit in channel $j$ of the input, with an offset of $k$ rows and $l$ columns between the output unit and the input unit. assume the input data is $V$ with element $V_{i,j,k}$ giving the value of the input unit w/n channel $i$ at row $j$ and column $k$. Assume output  $Z$ same format as $V$. If $Z$ is produced by convolving $K$ across $V$ w/o flipping $K$, then;<br>
$$Z_{i,j,k} = \sum_{l,m,n}V_{l,j+m-1,k+n-1} K_{i,l,m,n}$$
where the summation over $l, m, n$ is over all values for which the tensor indexing operations inside the summation is valid.<br>
![ch98](imgs/ch98.jpg)<br>
This is the convolution with a stride.<br>
Zero pading can allows us to control the kernel width and the size of the output independently.
![ch99](imgs/ch99.jpg)<br>
The effect of zero padding on network size.<br>